In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import analysis.yinf as yinf
import analysis.products as products
import analysis.util as util
from data.util import *
import os, sys, re
from scipy.stats import gaussian_kde
ML_DIR = os.path.expandvars("$SCRATCH/mlreco_cell/")
NETS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(ML_DIR))])))
SIM_DIR = os.path.expandvars("$SCRATCH/larsim/")
SIMS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(SIM_DIR)) if "reco" in d])))

def parse_stats_dir(stats_dir = './stats'):
    stats_files = []
    for root, dirs, files in os.walk(stats_dir):
        rel_path = os.path.relpath(root, stats_dir)
        if rel_path.count('/') != 1: continue
        slash_i = rel_path.find('/')
        header = [rel_path[:slash_i], rel_path[slash_i+1:]]
        for f in files:
            epoch, _ = inf_file_info(f)
            thres = re.findall('thres([0-9]*[.]?[0-9]+)', f)
            if len(thres) == 1: 
                thres = float(thres[0])
            else:
                thres = -1
            stats_files.append(header+[epoch, thres, root+'/'+f])
    return pd.DataFrame(stats_files, columns=['net', 'inf', 'epoch', 'thres', 'file'])

STATS = parse_stats_dir().sort_values(by=['epoch'])

def select_stats(net=None, inf=None, thres=None, first=0, last=-1, stride=1, stats_df=STATS):
    if net is not None:
        stats_df = stats_df[stats_df.net==net]
    if inf is not None:
        stats_df = stats_df[stats_df.inf==inf]
    if thres is not None:
        stats_df = stats_df[stats_df.thres==thres]
    if last == -1:
        epochs = sorted(set(stats_df.epoch))[first::stride]
    else:
        epochs = sorted(set(stats_df.epoch))[first:last+1:stride]
    return list(stats_df[stats_df.epoch.isin(epochs)]['file'])


In [ ]:
NETS

In [ ]:
SIMS

In [ ]:
def plot_train_summary(*net_dirs, start_epoch = 1):
    summaries = pd.DataFrame()
    for net_dir in net_dirs:
        print(net_dir)
        summary = pd.read_csv(ML_DIR+net_dir+'/summaries_0.csv', delim_whitespace=True)
        summary = summary.drop(columns=["epoch"]).drop(index=range(start_epoch))
        summaries[["train_loss: "+net_dir, "valid_loss: "+net_dir]] = summary[["train_loss", "valid_loss"]]
        try:
            summaries[["train_acc: "+net_dir, "valid_acc: "+net_dir]] = summary[["train_acc", "valid_acc"]]
        except:
            pass
        summaries[["train_time: "+net_dir, "valid_time: "+net_dir]] = summary[["train_time", "valid_time"]]
    summaries[[col for col in summaries.columns if '_loss' in col]].plot()
    try:
        summaries[[col for col in summaries.columns if '_acc' in col]].plot()
    except:
        pass
    summaries[[col for col in summaries.columns if '_time' in col]].plot()
    
def compute_yinf_stats(net_dir, xy_dir, start_epoch = 0, stride=1, tpc=None, evt_type=None, inf_dir="inference", **kwargs):
    stats_dir = "./stats/"+net_dir+'/'+inf_dir
    os.makedirs(stats_dir, exist_ok=True)
    print(net_dir)
    yinf_dir = ML_DIR+net_dir+'/'+inf_dir
    yinf_files = os.listdir(yinf_dir)
    if tpc is not None:
        yinf_files = [f for f in yinf_files if int(tpc_info(f)[0])==tpc]
    if evt_type is not None:
        yinf_files = [f for f in yinf_files if file_info(f)[0]==evt_type]
    yinf_epoch = files_info(yinf_files, [0], inf_file_info)
    n_epochs = len(yinf_epoch.keys())
    if start_epoch < 0:
        start_epoch += n_epochs
    for epoch in range(start_epoch, n_epochs, stride):
        yinf.yinf_avg_stats(epoch=epoch, tpc=tpc, evt_type=evt_type, stats_dir=stats_dir, xy_dir=SIM_DIR+xy_dir, yinf_dir=yinf_dir, **kwargs)

def find_n_last(text, pattern, n):
    if n==1: return text.rfind(pattern)
    return find_n_last(text[:text.rfind(pattern)], pattern, n-1)

def plot_yinf_stats(*stats_files):
    stats_files = np.concatenate(stats_files)
    labels = [f[find_n_last(f, '/', 3)+1:f.rfind('.')] for f in stats_files]
    yinf.compare_yinf_stats(plt.figure(), stats_files, labels)
    
def plot_mc_stats(stats_file, tpc=None, beamcosmic=None):
    if isinstance(stats_file, list):
        if tpc is None:
            stats_file = list(stats_file)[0]
        else:
            stats_file = [f for f in stats_file if "tpc%d_stats"%tpc in f][0]
    print(stats_file)
    filter_mc = lambda mc: mc[4]==beamcosmic if beamcosmic is not None else True
    yinf.plot_mc_stats(plt.figure(), plt.figure(), filter_mc, stats_file)
    
def plot_mc(xy_dir, mc_beamcosmic_dir=None, file_i=0, tpc=1, event=0, min_size=30, track_ids=None, pdg_ids=None, beamcosmic=None, plot_all=True, stats_file=None):
    xy_dir = SIM_DIR+xy_dir
    if mc_beamcosmic_dir is not None:
        mc_beamcosmic_dir = SIM_DIR+mc_beamcosmic_dir
    xy_file = sorted([f for f in os.listdir(xy_dir) if tpc_info(f)[0]==tpc])[file_i]
    def filter_mc(mc):
        if mc[2] < min_size: return False
        if track_ids is not None and mc[0] not in track_ids: return False
        if pdg_ids is not None and mc[1] not in pdg_ids: return False
        if beamcosmic is not None and mc[4]!=beamcosmic: return False
        return True
    products.plot_mc(plt.figure(), event, filter_mc, plot_all, mc_beamcosmic_dir, stats_file, xy_dir+'/'+xy_file)

def plot_yinf(figs, net_dir, xy_dir, epoch = -1, file_i=0, tpc=None, event = 0, downsample=(1,1,1), thres=None, true_thres=0, plot_slice=False, plot_lims=None, view_angle=None, inf_dir="inference"):
    fig, fig_hist, fig_inf, fig_hist_inf, fig_true, fig_true_hist, fig_true_inf, fig_true_hist_inf= figs
    yinf_dir = ML_DIR+net_dir+'/'+inf_dir
    yinf_files = os.listdir(yinf_dir)
    if tpc is not None:
        yinf_files = [f for f in yinf_files if int(tpc_info(f)[0])==tpc]
    yinf_epoch = files_info(yinf_files, [0], inf_file_info)
    epoch_i = sorted(yinf_epoch.keys())[epoch]
    yinf_file = sorted([f for f in yinf_epoch[epoch_i] if tpc is None or tpc_info(f)[0]==tpc])[file_i]
    plot_lims, thres = yinf.plot_yinf(fig, fig_hist, fig_inf, fig_hist_inf, 
                                      event, downsample, thres, true_thres, 
                                      plot_slice, plot_lims, view_angle, SIM_DIR+xy_dir, yinf_dir+'/'+yinf_file)
    products.compare_true_active(fig_true, fig_true_hist,  fig_true_inf, fig_true_hist_inf, 
                                 event, downsample, true_thres, 
                                 plot_slice, plot_lims, view_angle, yinf.get_xy_file(SIM_DIR+xy_dir, yinf_file))
    x_lim, y_lim, z_lim = plot_lims
    return x_lim, y_lim, z_lim, thres

def plot_true_v_inf(net_dir, xy_dir, epoch = -1, file_i=0, event = 0, inf_dir="inference"):
    yinf_dir = ML_DIR+net_dir+'/'+inf_dir
    yinf_epoch = files_info(os.listdir(yinf_dir), [0], inf_file_info)
    epoch_i = sorted(yinf_epoch.keys())[epoch]
    yinf_file = yinf_dir +'/'+ sorted(yinf_epoch[epoch_i])[file_i]
    print(yinf_file)
    xy_file = yinf.get_xy_file(SIM_DIR+xy_dir, yinf_file)
    
    voxel_truth, voxel_active, event_info = products.parse_xy(event, 1, xy_file)
    voxel_yinf = yinf.parse_yinf(event_info, yinf_file)
    _, intersect, FP = util.comp_voxels(voxel_truth, voxel_active, True)
    
    vals_T_active = np.array([[voxel_truth[coord], voxel_yinf[coord]] for coord in intersect])
    vals_ghosts = np.array([[-1, voxel_yinf[coord]] for coord in FP]).T
    dens_ghosts = gaussian_kde(vals_ghosts)(vals_ghosts)
    fig, axes = plt.subplots(2)
    ax, ax2 = axes
    ax.scatter(*vals_T_active.T, s=1)
    ax.scatter(*vals_ghosts, c = dens_ghosts, s=1)
    ax.set_xlabel("True Energy [MeV]")
    ax.set_title("Inference vs Energy")
    
    vals_T_active = np.array([[voxel_truth[coord], voxel_active[coord]] for coord in intersect])
    vals_ghosts = np.array([[-1, voxel_active[coord]] for coord in FP]).T
    dens_ghosts = gaussian_kde(vals_ghosts)(vals_ghosts)
    ax2.scatter(*vals_T_active.T, s=1)
    ax2.scatter(*vals_ghosts, c = dens_ghosts, s=1)
    ax2.set_xlabel("True Energy [MeV]")
    ax2.set_title("Pixel value vs Energy")
    
    fig.suptitle("Energy Scatter \nInference file: %s, Event: %d"%(yinf_file[yinf_file.rfind('/')+1:], event))

class PlotYinfWidget():   
    def __init__(self, out, **kwargs):
        if kwargs.get("event") is None:
            kwargs["event"] = 0
        self.view_angle = kwargs["view_angle"]
        kwargs["plot_slice"] = False
        self.figs = [plt.figure() for _ in range(8)]
        self.sigmoid=kwargs.pop("sigmoid", True)
        kwargs["figs"] = self.figs
        self.Size = self.figs[0].get_size_inches() 
        self.kwargs = kwargs
        self.out = out
    def get_widgets(self):
        bnext = widgets.Button(description='Next')
        bprev = widgets.Button(description='Previous')
        bset = widgets.Button(description='Set')
        breset = widgets.Button(description='Reset')
        bnext.on_click(callback.next)
        bprev.on_click(callback.prev)
        bset.on_click(callback.set)
        breset.on_click(callback.reset)
        self.xmin = widgets.IntSlider(
             step=1,
             description='xmin')
        self.xmax = widgets.IntSlider(
             step=1,
             description='xmax')
        self.ymin = widgets.IntSlider(
             step=1,
             description='ymin')
        self.ymax = widgets.IntSlider(
             step=1,
             description='ymax')
        self.zmin = widgets.IntSlider(
             step=1,
             description='zmin')
        self.zmax = widgets.IntSlider(
             step=1,
             description='zmax')
        self.thres = widgets.FloatSlider(
             min=0 if self.sigmoid else -10,
             max=1.0 if self.sigmoid else 10,
             step=0.01,
             description='threshold')
        self.elev = widgets.FloatSlider(
             value=self.view_angle[0],
             min=-90,
             max=90,
             step=1,
             description='elevation')
        self.azim = widgets.FloatSlider(
             value=self.view_angle[1],
             min=-90,
             max=90,
             step=1,
             description='azimuth')
        self.slice = widgets.Checkbox(
            value=False,
            description='2D slice')
        self.slice_invert = widgets.Checkbox(
            value=False,
            description='inverted 2D axis')
        self.plots = [widgets.Checkbox(
            value=True,
            description='Fig %d'%(i+self.figs[0].number)) for i in range(8)]
        self._plot_event()
        return bnext, bprev, bset, breset, \
            self.xmin, self.xmax, self.ymin, self.ymax, self.zmin, self.zmax, \
            self.thres, self.elev, self.azim, self.slice, self.slice_invert, self.plots
            
    def next(self, event):
        self.kwargs["event"] += 1
        self._clear_vals()
        self._plot_event() 
    def prev(self, event):
        if self.kwargs["event"]>0:
            self.kwargs["event"] -= 1
        self._clear_vals()
        self._plot_event()
    def set(self, event):
        self._read_vals()
        self._plot_event()
    def reset(self, event):
        self._clear_vals(True)
        self._plot_event()
    
    def _plot_event(self):
        for fig in self.figs:
            fig.clf()
        if not self.slice.value:
            self.xmin.max=self.xmax.max=10000
            self.xmin.min=self.xmax.min=-10000
        self.ymin.max=self.ymax.max=self.zmin.max=self.zmax.max=10000
        self.ymin.min=self.ymax.min=self.zmin.min=self.zmax.min=-10000
        with self.out:
            clear_output()
            defaults = plot_yinf(**self.kwargs)
            if self.slice.value:
                self.xmax.value = defaults[0][1]
                self.xmin.value = defaults[0][0]
            else:
                self.xmin.max=self.xmax.max=self.xmax.value =defaults[0][1]
                self.xmin.min=self.xmax.min=self.xmin.value =defaults[0][0]
            self.ymin.max=self.ymax.max=self.ymax.value = defaults[1][1]
            self.ymin.min=self.ymax.min=self.ymin.value = defaults[1][0]
            self.zmin.max=self.zmax.max=self.zmax.value = defaults[2][1]
            self.zmin.min=self.zmax.min=self.zmin.value = defaults[2][0]
            self.thres.value = defaults[3]
            self.kwargs["plot_lims"] = defaults[:3]
            self.kwargs["thres"] = defaults[3]

    def _read_vals(self):
        if self.slice.value and (self.kwargs["plot_lims"][0][0] != self.xmin.value or self.kwargs["plot_lims"][0][1] != self.xmax.value):
            if self.kwargs["plot_lims"][0][0] != self.xmin.value:
                self.xmax.value = self.xmin.value+1
            self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value), None, None)
        else:
            self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value),
                                        (self.ymin.value, self.ymax.value),
                                        (self.zmin.value, self.zmax.value))
        if self.thres.value != self.kwargs["thres"] or self.kwargs["plot_slice"] != self.slice.value:
            if self.slice.value:
                self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value), None, None)
            else:
                self.kwargs["plot_lims"] = None
                
        self.kwargs["plot_slice"] = self.slice.value        
        self.kwargs["thres"] = self.thres.value
        if self.slice.value:
            self.kwargs["view_angle"] = not self.slice_invert.value
        else:
            self.kwargs["view_angle"] = (self.elev.value, self.azim.value)
        self.kwargs["figs"] = [fig if plot.value else None for fig, plot in zip(self.figs, self.plots)]
        for fig, plot in zip(self.figs, self.plots):
            if plot.value:
                fig.set_size_inches(self.Size[0]*1.5, self.Size[1]*1.2, forward=True)
            else:
                fig.set_size_inches(self.Size[0]/3, self.Size[1]/3, forward=True)
    def _clear_vals(self, reset=False):
        if reset and self.slice.value:
            self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value), None, None)
            self.kwargs["view_angle"] = not self.slice_invert.value
        else:
            self.slice.value = self.kwargs["plot_slice"] = False
            self.kwargs["plot_lims"] = None
        self.kwargs["thres"] = None
        if self.slice.value:
            self.kwargs["view_angle"] = not self.slice_invert.value
        else:
            self.elev.value, self.azim.value = self.kwargs["view_angle"] = self.view_angle
        self.kwargs["figs"] = [fig if plot.value else None for fig, plot in zip(self.figs, self.plots)]
        for fig, plot in zip(self.figs, self.plots):
            if plot.value:
                fig.set_size_inches(self.Size[0]*1.5, self.Size[1]*1.2, forward=True)
            else:
                fig.set_size_inches(self.Size[0]/3, self.Size[1]/3, forward=True)


In [ ]:
TPC = 1
EVENT = 4
FILE = 10
TRACK = None
plot_mc('reco_1GeV_BeamCosmic_xy', None, beamcosmic="beam", file_i=FILE, event=EVENT, tpc=TPC, min_size=20, track_ids=TRACK, plot_all=False, stats_file="mc/reco_1GeV_BeamCosmic_xy-tpc%d-mc.npz"%TPC)
#plot_mc(SIMS[1], SIMS[0], pdg_ids=[2212, 11, -211, -11, 211],file_i=FILE, event=EVENT, tpc=TPC, min_size=20, plot_all=False, stats_file="mc/reco_1GeV_BeamCosmic_xy-tpc%d-mc.npz"%TPC)
plot_mc('reco_1GeV_BeamCosmic_xy', None, beamcosmic=None,file_i=FILE, event=EVENT, tpc=TPC, min_size=20, track_ids=TRACK, plot_all=False, stats_file="mc/reco_1GeV_BeamCosmic_xy-tpc%d-mc.npz"%TPC)
#plot_mc(SIMS[1], SIMS[0], beamcosmic=None,file_i=FILE, event=EVENT, tpc=TPC, min_size=20, plot_all=True, stats_file="mc/reco_1GeV_BeamCosmic_xy-tpc%d-mc.npz"%TPC)




In [ ]:
#plot_mc_stats(select_stats('singleElectronWire-v3_3-25k_ghost3D-164-lr0.01', "inf_cosmic", first=-1, thres=0.17), 1)
#plot_mc_stats('./stats/BeamCosmic-tpc159-crop_ghost3D-164-lr0.01/inference/epoch24-tpc5_stats-thres0.17-N500-v2.npz', None, "Beam")

#plot_mc_stats(select_stats('BeamCosmic-tpc5_ghost3D-164-lr0.01', 'inference', first=-1, thres=0.17), 5)
#plot_mc_stats("stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-BeamCosmic-tpc1_stats-thres0.17-N500-v2.npz", 1, "beam")
#plot_mc_stats(select_stats('singleE-BCtrained_ghost3D-164-lr0.01', 'inference', first=-1, thres=0.17), 1, "beam")
#products.mc_stats("/global/cscratch1/sd/ethanlu/larsim/reco_1GeV_BeamCosmic_parsed", 1, xy_dir = SIM_DIR+SIMS[1])
#products.plot_mc_stats(plt.figure(), "mc/reco_1GeV_BeamCosmicHit_xy-tpc1-mc.npz")
#plot_mc(SIMS[1], SIMS[0], beamcosmic=None, file_i=20, event=0, tpc=1, min_size=20, plot_all=False)


In [ ]:
stats_file = "stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-BeamCosmic-tpc1_stats-thres0.17-N500-all.npz"
MCs, AUCs, Sizes_true, Sizes_cluster, Indexes, Tracks, Purs_naive, Effs_naive, Thetas, Phis, Purs, Effs = [], [], [], [], [] ,[], [], [], [], [], [], []
E_true, E_baseline = [], []
N_true, N_baseline = [], []
PDG_id = [13, -13]
inf_thres = 25
i = 0
with np.load(stats_file, allow_pickle=True) as stats_f:
    while "arr_%d"%i in stats_f:
        mc_stats = stats_f["arr_%d"%i]
        i += 1
        mc, xs, ys, cluster_size, n_T_mc, n_TP_naive_mc, val_baseline, ind, evt = mc_stats
        if mc[1] in PDG_id: continue
        track_stats = mc[3]
        if track_stats is None: continue
        #if track_stats[0]>30 and track_stats[0]<150: continue
        MCs.append(mc)
        Thetas.append(track_stats[0])
        Phis.append(track_stats[1])
        AUCs.append(util.SP_score((xs, ys)))
        Sizes_true.append(mc[2])
        Sizes_cluster.append(cluster_size)
        Indexes.append((ind, evt))
        Tracks.append(mc[0])
        Effs_naive.append(n_TP_naive_mc/n_T_mc)
        Purs_naive.append(n_TP_naive_mc/cluster_size)
        Purs.append(xs[inf_thres])
        Effs.append(ys[inf_thres])
        E_true.append(mc[5])
        E_baseline.append(val_baseline)
        N_true.append(mc[2])
        N_baseline.append(n_T_mc)


In [ ]:
AUCs = np.array(AUCs)
Sizes_true = np.array(Sizes_true)
Sizes_cluster = np.array(Sizes_cluster)
Indexes = np.array(Indexes)
Tracks = np.array(Tracks)
Purs = np.array(Purs)
Effs = np.array(Effs)
E_baseline = np.array(E_baseline)
E_true = np.array(E_true)
N_baseline = np.array(N_baseline)
N_true = np.array(N_true)
MCs = np.array(MCs)
print(np.corrcoef(Purs_naive, AUCs))
args = np.argsort(N_true/E_true)[:20]
#args = Purs>0.2
print(AUCs[args])
print(Sizes_true[args])
print(Sizes_cluster[args])
print(Indexes[args])
print(Tracks[args])
print(Purs[args])
print(Effs[args])
print(MCs[args])

In [ ]:
fig, axes = plt.subplots(2)
axes[0].scatter(Sizes_true, AUCs, s=2)
axes[1].scatter(Sizes_cluster, AUCs, s=2)
fig, ax = plt.subplots()
cmap=plt.get_cmap('viridis')
fig.colorbar(ax.scatter(Purs_naive, Effs_naive, s=5, c=AUCs, cmap=cmap))

In [ ]:
from scipy import stats
fig, axes = plt.subplots(2)
bin_means, bin_edges, binnumber = stats.binned_statistic(Thetas, AUCs, bins=150)
axes[0].scatter(Thetas, AUCs, s=2)
axes[0].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
bin_means, bin_edges, binnumber = stats.binned_statistic(Phis, AUCs, bins=150)
axes[1].scatter(Phis, AUCs, s=2)
axes[1].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
axes[0].set_title("AUC vs. Theta")
axes[1].set_title("AUC vs. Phi")

fig, axes = plt.subplots(2)
bin_means, bin_edges, binnumber = stats.binned_statistic(Thetas, Effs, bins=150)
axes[0].scatter(Thetas, Effs, s=2)
axes[0].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
bin_means, bin_edges, binnumber = stats.binned_statistic(Phis, Effs, bins=150)
axes[1].scatter(Phis, Effs, s=2)
axes[1].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
axes[0].set_title("Efficiency vs. Theta")
axes[1].set_title("Efficiency vs. Phi")

fig, axes = plt.subplots(2)
bin_means, bin_edges, binnumber = stats.binned_statistic(Thetas, Purs, bins=150)
axes[0].scatter(Thetas, Purs, s=2)
axes[0].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
bin_means, bin_edges, binnumber = stats.binned_statistic(Phis, Purs, bins=150)
axes[1].scatter(Phis, Purs, s=2)
axes[1].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
axes[0].set_title("Purity vs. Theta")
axes[1].set_title("Purity vs. Phi")

In [ ]:
from scipy import stats
fig, axes = plt.subplots(2)
bin_means, bin_edges, binnumber = stats.binned_statistic(E_true, AUCs, bins=150)
axes[0].scatter(E_true, AUCs, s=2)
axes[0].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
bin_means, bin_edges, binnumber = stats.binned_statistic(E_baseline, AUCs, bins=150)
axes[1].scatter(E_baseline, AUCs, s=2)
axes[1].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
axes[0].set_title("AUC vs. Theta")
axes[1].set_title("AUC vs. Phi")
fig, ax=plt.subplots()
ax.scatter(E_true, E_baseline, s=2)
fig, ax=plt.subplots()
ax.scatter(Sizes_true, E_true, s=2)
print(np.mean(E_true-E_baseline))
print(np.mean((E_baseline)/E_true))
print(np.mean((N_baseline)/N_true))
print(np.sum(E_true-E_baseline)/np.sum(N_true-N_baseline))
print(np.corrcoef(E_true, AUCs))
print(np.corrcoef(E_true, Sizes_true))

In [ ]:
yinf_file = ML_DIR+'singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-batch0-reco_BeamCosmic_011_yinf-TPC1.npy'
xy_dir = SIM_DIR+"reco_1GeV_BeamCosmic_xy"
lims=((300, 700), None,None)
event=4
yinf.plot_yinf(plt.figure(), None, plt.figure(), None, 
    event=event, true_thres=.5/3/3, thres=0.35, 
    plot_slice=False, plot_lims=lims, xy_dir=xy_dir,
               yinf_file=yinf_file)
products.compare_true_active(plt.figure(), None, plt.figure(), None,  
                             event=event, true_thres=0,           
                         plot_slice=False, plot_lims=lims, xy_file=yinf.get_xy_file(xy_dir, yinf_file),)

In [ ]:
yinf_file = ML_DIR+'singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/filltruth/epoch24-batch0-reco_BeamCosmic_011_yinf-TPC1.npy'
xy_dir = SIM_DIR+"BeamCosmic_xy-filltruth"
lims=((300, 700), None,None)
event=4
yinf.plot_yinf(plt.figure(), None, plt.figure(), None, 
    event=event, true_thres=.5/3/3, thres=0.35, 
    plot_slice=False, plot_lims=lims, xy_dir=xy_dir,
               yinf_file=yinf_file)
products.compare_true_active(plt.figure(), None, plt.figure(), None,  
                             event=event, true_thres=0,           
                         plot_slice=False, plot_lims=lims, xy_file=yinf.get_xy_file(xy_dir, yinf_file),)

In [ ]:
products.plot_xy(plt.figure(), 4, yinf.get_xy_file(xy_dir, yinf_file))